In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Concatenate
from deepctr_torch.models.basemodel import build_input_features, get_linear_logit, input_from_feature_columns


pd.options.display.max_rows=10000 #Notebook 的一个cell的显示行数
pd.options.display.max_columns=10000#Notebook 的一个cell的显示列数

ImportError: cannot import name 'get_linear_logit' from 'deepctr_torch.models.basemodel' (/Users/wzq/anaconda3/envs/torch/lib/python3.8/site-packages/deepctr_torch/models/basemodel.py)

In [4]:
data = pd.read_csv('/Users/wzq/Desktop/coding/DeepCTR-Torch/examples/criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']


In [5]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
# 这里其实是将字符串等映射为数字，这样方便后续进行look up embedding
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [6]:
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [7]:
# 2.count #unique features for each sparse field,and record dense feature field name
# 生成一个了元素为SparseFeat类别的list，每个元素namedtuple：类似于一个轻量级的类，它允许创建具名字段的元组，可以像类一样使用。
# with signature：指的是创建这个 namedtuple 时的字段顺序和名称，也就是在实例化时需要按顺序传入这些字段的值。
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4)
                            for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                            for feat in dense_features]
fixlen_feature_columns

[SparseFeat(name='C1', vocabulary_size=27, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=92, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=172, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=157, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=12, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=183, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 SparseFeat

In [8]:
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,...,0,74,5,1,30,5,0,118,17,48
196,1,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,...,1,25,0,0,138,0,0,68,0,0
197,1,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,...,4,40,17,2,41,0,0,12,16,11
198,0,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,...,4,7,18,1,123,0,0,10,16,49


In [12]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
linear_feature_columns + dnn_feature_columns

[SparseFeat(name='C1', vocabulary_size=27, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=92, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=172, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=157, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=12, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=183, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 SparseFeat

In [25]:
build_input_features(dnn_feature_columns)

NameError: name 'build_input_features' is not defined

In [21]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

76     146
34      72
1       98
40      34
151    160
111    149
18     105
8      141
60      13
93      67
107    146
109    144
26      52
31     103
162    101
97      80
137     64
53      33
159     57
189      0
49      19
144    132
110     17
123     82
72     135
4       59
166     30
124     16
105     92
25      69
183    116
141     65
135    143
103     99
43     114
51      71
81      49
167    114
86       4
78     164
116    125
7       31
17     166
85      77
138    171
45      21
54      45
178     82
61      36
56      90
155    119
113     85
84       0
194     20
74      87
146     79
187    159
153      8
100    150
168    161
106    170
170    123
94      28
47     126
44      75
165    146
77       0
36     130
3        7
180    148
197    153
15      60
132     74
32     155
12     122
0       96
59       1
193     86
50     145
39      93
19      35
176    136
58     165
128    146
16     121
37      56
75       0
122     81
158    133
171     45
127    107

In [24]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",
                  metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  0.6897 - binary_crossentropy:  0.6897 - val_binary_crossentropy:  0.6806
Epoch 2/10
0s - loss:  0.6760 - binary_crossentropy:  0.6760 - val_binary_crossentropy:  0.6722
Epoch 3/10
0s - loss:  0.6623 - binary_crossentropy:  0.6623 - val_binary_crossentropy:  0.6640
Epoch 4/10
0s - loss:  0.6488 - binary_crossentropy:  0.6488 - val_binary_crossentropy:  0.6559
Epoch 5/10
0s - loss:  0.6356 - binary_crossentropy:  0.6356 - val_binary_crossentropy:  0.6482
Epoch 6/10
0s - loss:  0.6227 - binary_crossentropy:  0.6227 - val_binary_crossentropy:  0.6408
Epoch 7/10
0s - loss:  0.6099 - binary_crossentropy:  0.6099 - val_binary_crossentropy:  0.6335
Epoch 8/10
0s - loss:  0.5974 - binary_crossentropy:  0.5974 - val_binary_crossentropy:  0.6267
Epoch 9/10
0s - loss:  0.5851 - binary_crossentropy:  0.5851 - val_binary_crossentropy:  0.6201
Epoch 10/10
0s - loss:  0.5729 - binary_crossentropy:  0.5729 - val_